In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from nn.modules.conv import Conv, Concat
from nn.modules.block import C2f, SPPF
from nn.modules.head import Detect

class YOLOv8(nn.Module):
    def __init__(self):
        super().__init__()
        self.a0 = Conv(3,16,3,2)
        
        
    def forward(self, x):
        pass


ModuleNotFoundError: No module named 'nn.utils'

In [7]:
Conv(3,16,3,2)

Conv(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): SiLU()
)